# Exploring Genomic Data in Combination with HiGlass

In [ ]:
!curl -L -C - -o data/genomic-embeddings.pq https://storage.googleapis.com/flekschas/jupyter-scatter-tutorial/genomic-embeddings.pq
!curl -L -C - -o data/higlass-viewconfig.json https://storage.googleapis.com/flekschas/jupyter-scatter-tutorial/higlass-viewconfig.json

In [1]:
from __future__ import annotations

import pandas as pd
import numpy as np
import matplotlib.colors as colors
import jscatter
import ipywidgets
import traitlets

IPG_COLORS = {'A1': '#e23838', 'A2': '#f78200', 'AB': '#5ebd3e', 'A3': '#ffb900', 'B0': '#6495ED', 'B4': '#973999'}
IPG_COLORS_RGB = { k: f"{int(v[1:3], 16)},{int(v[3:5], 16)},{int(v[5:7], 16)}" for k, v in IPG_COLORS.items() }

def hex_to_rgb(hex_code: str):
    if len(hex_code) != 7 or hex_code[0] != '#':
        raise ValueError('Input should be a 7-character string in the form #XXXXXX')
    r = int(hex_code[1:3], 16)
    g = int(hex_code[3:5], 16)
    b = int(hex_code[5:7], 16)
    return f"{r},{g},{b}"

def load_stages():
    df = pd.read_parquet("data/genomic-embeddings.pq")
    joint = df.iloc[:, ~df.columns.str.match("^stage\..*$")]
    return joint, dict(iter_stages(df))

def iter_stages(data: pd.DataFrame, stages: tuple[str, ...] = ("ESC", "DE", "HB", "iHEP", "mHEP")):
    metadata = data.iloc[:, ~data.columns.str.match("^stage\..*$")]
    # metadata = metadata.drop("IPG", axis=1)
    for stage in stages:
        df = data.filter(regex=f"^stage\.{stage}\..*$")
        df.columns = df.columns.str.removeprefix(f"stage.{stage}.")
        df = df.drop("IPG", axis=1)
        yield stage, metadata.join(df, how="right")

def color_kwargs(series: pd.Series):
    defaults = {
        "GC": { "norm": colors.Normalize(vmin=0.35, vmax=0.65), "map": "RdYlBu_r" },
        "centel_abs": { "norm": colors.Normalize(vmin=0, vmax=149043529), "map": "Greys" },
        "IPG": { "map": IPG_COLORS }
    }
    if series.name in defaults:
        return defaults[series.name]
    if pd.api.types.is_categorical_dtype(series):
        return {
            "map": dict(zip(series.cat.categories, jscatter.glasbey_dark)),
        }
    if pd.api.types.is_numeric_dtype(series):
        return {
            "norm": colors.Normalize(vmin=series.min(), vmax=series.max()),
            "map": "viridis_r",
        }
    return {}

In [2]:
def init_scatters(stages: dict[str, pd.DataFrame], x: str = "E1", y: str = "E2", color: str = "IPG"):
    scatters = [jscatter.Scatter(x=x, y=y, data=data, opacity=0.5) for data in stages.values()]
    dropdowns = init_dropdowns(x=x, y=y, color=color, scatters=scatters)
    linked_scatters = jscatter.compose(scatters, sync_selection=True)
    # TODO: remove the following and replace with below after next release 
    # jscatter.compose(list(zip(stages, scatters)), sync_selection=True),
    linked_scatters.children = [
        ipywidgets.VBox([ipywidgets.Label(stage), scatter])
        for stage, scatter in zip(stages, linked_scatters.children)
    ]
    linked_scatters.add_traits(
        coords=traitlets.Any(pd.DataFrame(columns=["chrom", "start", "end"]))
    )
    ipywidgets.dlink(
        source=(scatters[0].widget, "selection"),
        target=(linked_scatters, "coords"),
        transform=lambda ind: scatters[0]._data.iloc[ind][["chrom", "start", "end"]],
    )
    return ipywidgets.VBox([linked_scatters, ipywidgets.HBox(dropdowns)])

def init_dropdowns(x: str, y: str, color: str, scatters: list[jscatter.Scatter]):
    _stage = scatters[0]._data
    is_eigenvector_column = _stage.columns.str.match(r"^E\d$")
    xy_options = _stage.columns[is_eigenvector_column]
    x_dropdown = ipywidgets.Dropdown(options=xy_options, value=x, description="x:")
    def on_change_x(change):
        for scatter in scatters:
            scatter.x(change.new)
    y_dropdown = ipywidgets.Dropdown(options=xy_options, value=y, description="y:")
    def on_change_y(change):
        for scatter in scatters:
            scatter.y(change.new)
    color_options = reversed(
        list(c for c in _stage.columns[~is_eigenvector_column] if c not in ("start", "end"))
    )
    c_dropdown = ipywidgets.Dropdown(options=color_options, value=color, description="color:")
    def on_change_color(change):
        for scatter in scatters:
            scatter.color(by=change["new"], **color_kwargs(scatter._data[change["new"]]))

    x_dropdown.observe(on_change_x, names=["value"])
    y_dropdown.observe(on_change_y, names=["value"])
    c_dropdown.observe(on_change_color, names=["value"])
    on_change_color(dict(new=color))

    return x_dropdown, y_dropdown, c_dropdown

In [5]:
%pip install higlass bioframe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 2.8 MB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 4.4 MB/s eta 0:00:004.5 MB/s eta 0:00:01
  Using cached rich-13.4.2-py3-none-any.whl (239 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.9/336.9 kB 2.1 MB/s eta 0:00:002.2 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
  Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 7.9 MB/s eta 0:00:00
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
Note: you may need to resta

In [8]:
import dataclasses
import higlass as hg
import bioframe as bf
import uuid

def iter_bedtiles(df: pd.DataFrame, chrom_starts: dict[str, int]):
    df = bf.cluster(df).groupby("cluster").agg({"chrom": "first", "start": "min", "end": "max", "IPG": "first"})
    for (chrom, start, end, ipg) in df.values:
        start, end = int(start), int(end)
        yield {
            "chrOffset": chrom_starts[chrom],
            "xStart": chrom_starts[chrom] + start,
            "xEnd": chrom_starts[chrom] + end,
            "importance": 0,
            "uid": uuid.uuid4().hex,
            "fields": (chrom, start, end, ipg, ".", ".", start, end, IPG_COLORS_RGB[ipg])
        }
    
@dataclasses.dataclass
class DynamicTileset:
    name: str
    chromsizes: pd.Series
    uid: str = dataclasses.field(default_factory=lambda: uuid.uuid4().hex)
    datatype: str = "bedlike"
        
    def __post_init__(self):
        abslen = self.chromsizes.cumsum()
        starts = pd.Series([0] + abslen[:-1].tolist(), index=abslen.index)
        self._starts =  {k: int(v) for k, v in starts.items()}
        self._tiles = []
        
    def update(self, df: pd.DataFrame):
        self._tiles = list(iter_bedtiles(df, self._starts))
        
    def info(self):
        total = int(np.sum(self.chromsizes.values))
        return {"uuid": self.uid, "max_width": total, "min_pos": [1], "max_pos": [total], "max_zoom": 0}
     
    def tiles(self, _tileids):
        return [(f"{self.uid}.0.0", self._tiles)]

def init_dynamic_track(bins: pd.DataFrame):
    tileset = DynamicTileset(name="IPG clusters", chromsizes=bf.fetch_chromsizes('hg38')[:'chrY'])
    track = hg.server.add(tileset).track(height=30).opts(fillOpacity=1)
    def update_tileset(coords):
        if len(coords) > 0:
            inds = bf.overlap(bins, coords, how="inner", return_index=True, return_input=False)
            tileset.update(bins.loc[inds["index"], :])
        else:
            tileset.update(bins)

    update_tileset([])
    return track, update_tileset

def init_viewer(
    scatters: traitlets.HasTraits,
    bins: pd.DataFrame,
):
    conf = hg.Viewconf.parse_file("data/higlass-viewconf.json")
    track, update_tileset = init_dynamic_track(bins)
    view = conf.views[0]
    view.tracks.top[1].height= 100
    view.tracks.top.append(track)
    
    hg_viewer = conf.widget()
    
    def on_coords_change(change):
        update_tileset(change.new)
        hg_viewer.reload(dict(trackId=track.uid, viewId=view.uid))

    scatters.children[0].observe(on_coords_change, names=["coords"])
    return hg_viewer

In [9]:
joint, stages = load_stages()
scatters = init_scatters(stages=stages)
viewer = init_viewer(scatters=scatters, bins=joint)

ipywidgets.VBox([viewer, scatters])